In [4]:
import pandas as pd

In [77]:
# ---------------- core config ----------------
REQUIRED_COLS = ["classe","day","#day","plage","start","end","semaine","cours","pro","local","code horaire"]  # adjust to your exact headers

PLAGE_MAP = {"P00":"P0", "P01":"P1"}  # centralized normalization

TEMPLATE_NAME_BY_GROUP = {
    # You said: classes use "template P25", teachers use "template P025"
    "classe": {"P0": "template P0", "P25": "template P25", "P025": "template P25", "default": "template"},
    "pro":    {"P0": "template P0", "P25": "template P025", "P025": "template P025", "default": "template"},
}

# ---------------- helpers ----------------
import re, string
from collections import defaultdict
from pathlib import Path
import openpyxl
from openpyxl.utils import get_column_letter

def normalize_plage(v: str) -> str:
    v = str(v).strip().upper()
    return PLAGE_MAP.get(v, v)

def safe_title(name: str, taken: set) -> str:
    # Excel rules: max 31 chars, cannot contain: : \ / ? * [ ]
    cleaned = re.sub(r'[:\\/\?\*\[\]]', '_', name).strip()
    cleaned = cleaned[:31] or "Sheet"
    base = cleaned
    i = 2
    while cleaned in taken:
        suffix = f" ({i})"
        cleaned = (base[:31-len(suffix)] + suffix)
        i += 1
    taken.add(cleaned)
    return cleaned

def make_tags_map(ws):
    """Scan once per template. Returns {tag -> (row, col)}"""
    tags = {}
    for row in ws.iter_rows(values_only=False):
        for cell in row:
            v = cell.value
            if isinstance(v, str) and "{" in v and "}" in v:
                # crude but effective: treat each {...} as one tag
                for m in re.finditer(r"\{[^}]+\}", v):
                    tags[m.group(0)] = (cell.row, cell.column)
    return tags

def pick_template(rows_df, group_field: str):
    # decide the template by the presence of a normalized plage in the group
    plages = {normalize_plage(p) for p in rows_df["plage"].astype(str)}
    priority = ["P25", "P025", "P0"]  # pick the most specific present
    for p in priority:
        if p in plages:
            return TEMPLATE_NAME_BY_GROUP[group_field].get(p, TEMPLATE_NAME_BY_GROUP[group_field]["default"])
    return TEMPLATE_NAME_BY_GROUP[group_field]["default"]

def merge_ab(ws, tags_map, row):
    """
    If semaine == 'AB', merge the A/B cells for c,p,o,l of the given slot.
    Also merge when both A and B cells are empty (placeholders cleared).
    """
    # normalize plage like before (P00->P0, P01->P1)
    raw_plage = str(row["plage"])
    if raw_plage == "P00":
        norm_plage = "P0"
    elif raw_plage == "P01":
        norm_plage = "P1"
    else:
        norm_plage = raw_plage

    dnum = int(row["#day"])
    t_a = f"{{j{dnum}{norm_plage.lower()}a"
    t_b = f"{{j{dnum}{norm_plage.lower()}b"

    for suf in ["c", "p", "o", "l"]:
        tag_a = t_a + suf + "}"
        tag_b = t_b + suf + "}"
        if tag_a in tags_map and tag_b in tags_map:
            r1, c1 = tags_map[tag_a]
            r2, c2 = tags_map[tag_b]

            # Look at current cell contents (could be "", None, or text)
            v_a = ws.cell(r1, c1).value
            v_b = ws.cell(r2, c2).value
            both_empty = (v_a is None or str(v_a).strip() == "") and (v_b is None or str(v_b).strip() == "")

            # Merge if we have an AB slot either way — populated or both empty
            # (keeps your previous behavior, and now also joins blank A/B cells)
            ws.merge_cells(
                start_row=min(r1, r2),
                start_column=min(c1, c2),
                end_row=max(r1, r2),
                end_column=max(c1, c2),
            )


def fill_sheet(ws, tags_map, group_rows):
    """
    - Populate tags for each row
    - 'o' tag = join of all 'code horaire' for same (#day, plage_norm, semaine)
    - Clear any tags that were not populated (leftover placeholders)
    """
    filled = set()  # track which tags we wrote

    # set title tag
    if "{title}" in tags_map:
        rr, cc = tags_map["{title}"]
        ws.cell(rr, cc).value = ws.title
        filled.add("{title}")
    
    # Precompute a normalized-plage column locally to mirror how we build tag stems
    rows = group_rows.copy()
    rows["plage_norm"] = rows["plage"].apply(normalize_plage)

    for _, r in rows.iterrows():
        # normalize plage exactly like your snippet (P00->P0, P01->P1)
        raw_plage = str(r["plage"])
        if raw_plage == "P00":
            norm_plage = "P0"
        elif raw_plage == "P01":
            norm_plage = "P1"
        else:
            norm_plage = raw_plage

        # sem char used in tag stem: use r['sem'] if present, else derive from 'semaine'
        if "sem" in rows.columns:
            sem_char = str(r["sem"])
        else:
            sem_full = str(r["semaine"]).strip().upper()
            sem_char = "a" if sem_full in ("A", "AB") else "b"

        tag_stem = f"{{j{int(r['#day'])}{norm_plage.lower()}{sem_char}"

        # 'o' tag: join all code horaire for same day/plage_norm/semaine within this group
        same_slot = rows[
            (rows["#day"] == r["#day"]) &
            (rows["plage_norm"] == norm_plage) &
            (rows["semaine"] == r["semaine"])
        ]["code horaire"].astype(str)
        o_value = " ; ".join(same_slot)

        def set_tag(suffix, value):
            tag = tag_stem + suffix
            if tag in tags_map:
                rr, cc = tags_map[tag]
                ws.cell(rr, cc).value = value
                filled.add(tag)

        # exactly your mapping
        set_tag("c}", str(r["cours"]).upper())
        set_tag("p}", r["pro"])
        set_tag("l}", r["local"])
        set_tag("o}", o_value)

        # Merge if AB
        if str(r["semaine"]).strip().upper() == "AB":
            merge_ab(ws, tags_map, r)

    GRAY_FILL = PatternFill(start_color="C0C0C0", end_color="C0C0C0", fill_type="solid")
    
    # Cleanup: clear any leftover tag placeholders that weren't populated
    for tag, (rr, cc) in tags_map.items():
        if tag not in filled:
            val = ws.cell(rr, cc).value
            if isinstance(val, str) and val.strip().startswith("{") and val.strip().endswith("}"):
                ws.cell(rr, cc).value = ""
                ws.cell(rr, cc).fill = GRAY_FILL
            elif val is None:
                ws.cell(rr, cc).fill = GRAY_FILL
            else:
                # If you want to gray-fill regardless of placeholder detection:
                ws.cell(rr, cc).fill = GRAY_FILL

# ---------------- main builder ----------------
def build_workbook(df, group_field: str, template_path: str, out_path: str, logo_path: str | None = None):
    """
    group_field: 'classe' or 'pro'
    """
    # 1) validate + normalize
    missing = [c for c in REQUIRED_COLS if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}")
    df = df.copy()
    df["plage"] = df["plage"].apply(normalize_plage)

    # sort for deterministic output
    groups = sorted(df[group_field].dropna().unique(), key=str.casefold)

    # Load the template AS the output workbook (so we can copy within it)
    out_wb = openpyxl.load_workbook(template_path, data_only=False, keep_vba=False)

    # Cache tag maps per template sheet (scan once)
    tag_cache = {}
    used_titles = set()

    for g in groups:
        rows = df[df[group_field] == g]
        if rows.empty:
            continue

        template_name = pick_template(rows, group_field)
        if template_name not in out_wb.sheetnames:
            raise KeyError(f"Template '{template_name}' not found in template file.")

        # Copy template sheet within the SAME workbook
        tpl_ws = out_wb[template_name]
        new_ws = out_wb.copy_worksheet(tpl_ws)
        new_ws.title = safe_title(str(g), used_titles)

        # Tag map for this template (cached)
        if template_name not in tag_cache:
            tag_cache[template_name] = make_tags_map(tpl_ws)
        tags_map = tag_cache[template_name]

        # (Optional) re-add logo here if your template copy doesn’t include images
        if logo_path and Path(logo_path).exists():
            from openpyxl import drawing 
            img = drawing.image.Image('../templates/Logo Ecole .jpg')
            scale = 0.1
            img.width = img.width * scale
            img.height = img.height * scale
            new_ws.add_image(img, "A1")

        # Fill with data
        fill_sheet(new_ws, tags_map, rows)

    # Remove the template tabs (leave only generated class/teacher sheets)
    for name in list(out_wb.sheetnames):
        if name.lower().startswith("template"):
            out_wb.remove(out_wb[name])

    out_file = Path(out_path)
    out_file.parent.mkdir(parents=True, exist_ok=True)
    out_wb.save(out_file)
    print(f"✓ Saved {len(out_wb.sheetnames)} sheets → {out_file}")


In [78]:
df = pd.read_excel("../input/inputSM_2025-2026.xlsx", "input")

In [79]:
# df = ...  # your big dataframe, already read from the “input” sheet
build_workbook(
    df=df,
    group_field="classe",
    template_path="../templates/schedule_template.xlsx",
    out_path="../output/horaire_classes-v2.xlsx",
    logo_path="../templates/Logo Ecole .jpg",
)

build_workbook(
    df=df,
    group_field="pro",
    template_path="../templates/schedule_template.xlsx",
    out_path="../output/horaire_pros-v2.xlsx",
    logo_path="../templates/Logo Ecole .jpg",
)

✓ Saved 26 sheets → ../output/horaire_classes-v2.xlsx
✓ Saved 48 sheets → ../output/horaire_pros-v2.xlsx
